In [ ]:
# In this project I will create a Deep learning regression model that is capable 
# of predicting the percent chance of admission for students applying to graduate school

# admissions_data.csv provided by codecademy

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.layers import InputLayer

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('admissions_data.csv')

print(data.head())

   Serial No.  GRE Score  TOEFL Score  University Rating  SOP  LOR   CGPA  \
0           1        337          118                  4  4.5   4.5  9.65   
1           2        324          107                  4  4.0   4.5  8.87   
2           3        316          104                  3  3.0   3.5  8.00   
3           4        322          110                  3  3.5   2.5  8.67   
4           5        314          103                  2  2.0   3.0  8.21   

   Research  Chance of Admit   
0         1              0.92  
1         1              0.76  
2         1              0.72  
3         1              0.80  
4         0              0.65  


In [3]:
print(data.columns)

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')


In [4]:
features = data.iloc[:, 0:-1]
labels = data['Chance of Admit ']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state = 42)

In [6]:
scaler = StandardScaler()
features_train_scale = scaler.fit_transform(X_train)
features_test_scale = scaler.transform(X_test)

In [7]:
from tensorflow.keras.layers import Dense
my_model = Sequential()

input = InputLayer(input_shape = features_train_scale.shape[1])

my_model.add(input)
my_model.add(Dense(64,activation = 'relu'))
my_model.add(Dense(8,activation = 'relu'))
my_model.add(Dense(1))

In [8]:
Sequential.summary(my_model)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 8)                 520       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1,105
Trainable params: 1,105
Non-trainable params: 0
_________________________________________________________________


In [9]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.01, name='Adam')
my_model.compile(optimizer = opt, loss="mse", metrics=["mae"])

In [10]:
my_model.fit(features_train_scale, y_train, epochs = 40, batch_size =1,verbose = 1)

Epoch 1/40
400/400 [==============================] - 3s 3ms/step - loss: 0.0567 - mae: 0.1576
Epoch 2/40
400/400 [==============================] - 1s 3ms/step - loss: 0.0069 - mae: 0.0645
Epoch 3/40
400/400 [==============================] - 1s 2ms/step - loss: 0.0046 - mae: 0.0527
Epoch 4/40
400/400 [==============================] - 1s 2ms/step - loss: 0.0044 - mae: 0.0514
Epoch 5/40
400/400 [==============================] - 1s 2ms/step - loss: 0.0041 - mae: 0.0491
Epoch 6/40
400/400 [==============================] - 1s 3ms/step - loss: 0.0040 - mae: 0.0467
Epoch 7/40
400/400 [==============================] - 2s 4ms/step - loss: 0.0043 - mae: 0.0496
Epoch 8/40
400/400 [==============================] - 2s 4ms/step - loss: 0.0050 - mae: 0.0566
Epoch 9/40
400/400 [==============================] - 2s 4ms/step - loss: 0.0050 - mae: 0.0551
Epoch 10/40
400/400 [==============================] - 2s 4ms/step - loss: 0.0045 - mae: 0.0512
Epoch 11/40
400/400 [============================

In [21]:
res_mse, res_mae = my_model.evaluate(features_test_scale,y_test,verbose = 0)

In [22]:
print(res_mse**.5, res_mae)

0.08298896126705316 0.0677882507443428


In [39]:
def func():
    my_model = Sequential()
    input = InputLayer(input_shape = features_train_scale.shape[1])
    my_model.add(input)
    my_model.add(Dense(64,activation = 'relu'))
    my_model.add(Dense(8,activation = 'relu'))
    my_model.add(Dense(1))
    opt = Adam(learning_rate=0.01, name='Adam')
    my_model.compile(optimizer = opt, loss="mse", metrics=["mae"])
    return my_model

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

In [ ]:
# Grid Search automated tuning for batch size and epochs

In [71]:
def do_randomized_search():
  param_grid = {'batch_size': sp_randint(1, 16), 'epochs': sp_randint(10, 100)}
  model = KerasRegressor(model=func, verbose=0)
  grid = RandomizedSearchCV(estimator = model, param_distributions=param_grid, scoring = make_scorer(mean_squared_error, greater_is_better=False), n_iter = 12)
  grid_result = grid.fit(features_train_scale, y_train, verbose = 0)
  print(grid_result)
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  for mean, stdev, param in zip(means, stds, params):
      print("%f (%f) with: %r" % (mean, stdev, param))

In [72]:
do_randomized_search()

RandomizedSearchCV(estimator=KerasRegressor(model=<function func at 0x000001E5881D3EE0>, verbose=0),
                   n_iter=12,
                   param_distributions={'batch_size': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E597944190>,
                                        'epochs': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E5894231F0>},
                   scoring=make_scorer(mean_squared_error, greater_is_better=False))
Best: -0.004002 using {'batch_size': 11, 'epochs': 72}
-0.004701 (0.000862) with: {'batch_size': 10, 'epochs': 97}
-0.004933 (0.001133) with: {'batch_size': 7, 'epochs': 12}
-0.005080 (0.000288) with: {'batch_size': 3, 'epochs': 93}
-0.004872 (0.000627) with: {'batch_size': 5, 'epochs': 89}
-0.004613 (0.000966) with: {'batch_size': 2, 'epochs': 35}
-0.004097 (0.000618) with: {'batch_size': 2, 'epochs': 20}
-0.005562 (0.000600) with: {'batch_size': 11, 'epochs': 19}
-0.005033 (0.000658) with: {'batch_size': 1, 'epochs': 25

In [ ]:
# Using a grid search to tune batch size and epochs using knowledge from the random search

In [ ]:
from sklearn.model_selection import GridSearchCV

In [68]:
def do_grid_search():    
  batch_size = [10, 15,20,]
  epochs = [10, 30,35]
  model = KerasRegressor(model=func, verbose=0)
  param_grid = dict(batch_size=batch_size, epochs=epochs)
  grid = GridSearchCV(estimator = model, param_grid=param_grid, scoring = make_scorer(mean_squared_error, greater_is_better=False),return_train_score = True)
  grid_result = grid.fit(features_train_scale, y_train, verbose = 0)
  print(grid_result)
  print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

  means = grid_result.cv_results_['mean_test_score']
  stds = grid_result.cv_results_['std_test_score']
  params = grid_result.cv_results_['params']
  for mean, stdev, param in zip(means, stds, params):
      print("%f (%f) with: %r" % (mean, stdev, param))

  print("Traininig")
  means = grid_result.cv_results_['mean_train_score']
  stds = grid_result.cv_results_['std_train_score']
  for mean, stdev, param in zip(means, stds, params):
      print("%f (%f) with: %r" % (mean, stdev, param))

In [69]:
do_grid_search()

GridSearchCV(estimator=KerasRegressor(model=<function func at 0x000001E5881D3EE0>, verbose=0),
             param_grid={'batch_size': [10, 15, 20], 'epochs': [10, 30, 35]},
             return_train_score=True,
             scoring=make_scorer(mean_squared_error, greater_is_better=False))
Best: -0.003997 using {'batch_size': 20, 'epochs': 30}
-0.004479 (0.000725) with: {'batch_size': 10, 'epochs': 10}
-0.004100 (0.000533) with: {'batch_size': 10, 'epochs': 30}
-0.004582 (0.000936) with: {'batch_size': 10, 'epochs': 35}
-0.004835 (0.000567) with: {'batch_size': 15, 'epochs': 10}
-0.004882 (0.000708) with: {'batch_size': 15, 'epochs': 30}
-0.004781 (0.001239) with: {'batch_size': 15, 'epochs': 35}
-0.005296 (0.000757) with: {'batch_size': 20, 'epochs': 10}
-0.003997 (0.000271) with: {'batch_size': 20, 'epochs': 30}
-0.004744 (0.000832) with: {'batch_size': 20, 'epochs': 35}
Traininig
-0.002808 (0.000441) with: {'batch_size': 10, 'epochs': 10}
-0.002032 (0.000457) with: {'batch_size': 10,

In [95]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=20)
my_model.fit(features_train_scale, y_train, epochs = 30, batch_size =20,verbose = 0,callbacks = [es])

In [96]:
res_mse, res_mae = my_model.evaluate(features_test_scale,y_test,verbose = 0)

In [97]:
#The new model has a root-mean-square deviation of .067 and a mean absolute error of .050
print(res_mse**.5, res_mae)

0.07564701611559317 0.05681334808468819


In [100]:
predicted_values = my_model.predict(features_test_scale) 
print(r2_score(y_test, predicted_values)) 

4/4 [==============================] - 0s 1ms/step
0.7201725990823444


In [ ]:
# R squared value of .72 means that 72 percent of the variation in the chance of admission is accounted for by my model.

In [101]:
# Conclusion: My model presents the factors that can help students gain admission to graduate school
# By utilizing these findings, students and test prep companies can statigize more efficiently